In [1]:
from __future__ import print_function

import os
import argparse
import time
import numpy as np
import mxnet as mx
import h5py
from mxnet import nd,gluon,autograd
from mxnet.gluon import nn,data as gdata
import pandas as pd
from d2l import mxnet as d2l

from visualization.util_vtk import visualization
from dataset import ShapeNet3D
from model import BlockOuterNet,Conv3DNet
from criterion import BatchIoU
from misc import decode_multiple_block, execute_shape_program
from interpreter import Interpreter
from programs.loop_gen import translate, rotate, end
from options import options_guided_adaptation,options_train_generator,options_train_executor
import socket

In [2]:
def parse_argument():

    parser = argparse.ArgumentParser(description="testing the program generator")

    parser.add_argument('--model', type=str, default='model/generator of GA on shapenet {}',
                        help='path to the testing model')
    parser.add_argument('--data', type=str, default='./data/{}_testing.h5',
                        help='path to the testing data')
    parser.add_argument('--save_path', type=str, default='./output/{}/',
                        help='path to save the output results')
    parser.add_argument('--batch_size', type=int, default=32, help='batch size')
    parser.add_argument('--num_workers', type=int, default=4, help='number of workers')
    parser.add_argument('--info_interval', type=int, default=10, help='freq for printing info')

    parser.add_argument('--save_prog', action='store_true', help='save programs to text file')
    parser.add_argument('--save_img', action='store_true', help='render reconstructed shapes to images')
    parser.add_argument('--num_render', type=int, default=10, help='how many samples to be rendered')

    #parser = get_parser()
    opt,unknown = parser.parse_known_args()

    opt.prog_save_path = os.path.join(opt.save_path, 'programs')
    opt.imgs_save_path = os.path.join(opt.save_path, 'images')


    return opt


def test_on_shapenet_data(epoch, test_loader, model,model_wo, opt, ctx, gen_shape=False):

    generated_shapes = []
    generated_shapes_wo = []
    original_shapes = []
    gen_pgms = []
    gen_params = []

    for idx, data in enumerate(test_loader):
        start = time.time()

        shapes = data
        shape = nd.expand_dims(data,axis = 1).as_in_context(ctx)
        with autograd.train_mode():
            out = model.decode(shape)
            out_wo = model_wo.decode(shape)

        end = time.time()
        out[0] = nd.round(out[0]).astype('int64')
        out[1] = nd.round(out[1]).astype('int64')
        out_wo[0] = nd.round(out_wo[0]).astype('int64')
        out_wo[1] = nd.round(out_wo[1]).astype('int64')
        if gen_shape:
            generated_shapes.append(decode_multiple_block(out[0], out[1]))
            generated_shapes_wo.append(decode_multiple_block(out_wo[0], out_wo[1]))
            original_shapes.append(data.asnumpy())
            save_pgms = nd.argmax(out[0], axis=3).asnumpy()
            save_params = out[1].asnumpy()
            gen_pgms.append(save_pgms)
            gen_params.append(save_params)

        if idx % opt.info_interval == 0:
            print("Test: epoch {} batch {}/{}, time={:.3f}".format(epoch, idx, len(test_loader), end - start))

    if gen_shape:
        generated_shapes = np.concatenate(generated_shapes, axis=0)
        generated_shapes_wo = np.concatenate(generated_shapes_wo, axis=0)
        original_shapes = np.concatenate(original_shapes, axis=0)
        gen_pgms = np.concatenate(gen_pgms, axis=0)
        gen_params = np.concatenate(gen_params, axis=0)

    return original_shapes, generated_shapes,generated_shapes_wo,gen_pgms, gen_params



    


In [3]:
opt = parse_argument()



print('========= arguments =========')
for key, val in vars(opt).items():
    print("{:20} {}".format(key, val))
print('========= arguments =========')

def visual(path,gen_shapes,file_name,nums_samples):
    data = gen_shapes.transpose((0, 3, 2, 1))
    data = np.flip(data, axis=2)
    num_shapes = data.shape[0]
    for i in range(min(nums_samples,num_shapes)):
        voxels = data[i]
        save_name = os.path.join(path, file_name.format(i))
        visualization(voxels,
                      threshold=0.1,
                      save_name=save_name,
                      uniform_size=0.9)
def test_on_shape_net(tp,model_path, data_path,save_path,num_img):
    # data loader
    test_set = ShapeNet3D(data_path)
    test_loader = gdata.DataLoader(
        dataset=test_set,
        batch_size=opt.batch_size,
        shuffle=False,
        num_workers=opt.num_workers,
    )
    
    # model
    ctx = d2l.try_gpu()
    opt_gen = options_train_generator.parse()
    model = BlockOuterNet(opt_gen)
    model_wo = BlockOuterNet(opt_gen)
    model.init_blocks(ctx)
    model_wo.init_blocks(ctx)
    model_wo.load_parameters("model/model of blockouternet")
    model.load_parameters(model_path)


    # test the model and evaluate the IoU
    ori_shapes, gen_shapes,gen_shapes_wo ,pgms, params = test_on_shapenet_data(epoch=0,
                                                                 test_loader=test_loader,
                                                                 model=model,
                                                                 model_wo = model_wo,
                                                                 opt=opt,ctx=ctx,
                                                                 gen_shape=True)
    
        # Visualization
    visual(save_path,gen_shapes,'GA {}.png',num_img)
    visual(save_path,ori_shapes,'GT {}.png',num_img)
    visual(save_path,gen_shapes_wo,'Before GA {}.png',num_img)
    gen_shapes = nd.from_numpy(gen_shapes).astype('float32')
    ori_shapes = nd.from_numpy(ori_shapes).astype('float32')
    gen_shapes_wo = nd.from_numpy(gen_shapes_wo).astype('float32')
    IoU1 = BatchIoU(ori_shapes.copy(), gen_shapes)
    IoU2 = BatchIoU(ori_shapes.copy(), gen_shapes_wo)
    
    # execute the generated program to generate the reconstructed shapes
    # for double-check purpose, can be disabled
    '''
    num_shapes = gen_shapes.shape[0]
    res = []
    for i in range(num_shapes):
        data = execute_shape_program(pgms[i], params[i])
        res.append(data.reshape((1, 32, 32, 32)))
    res = nd.from_numpy(np.concatenate(res, axis=0)).astype('float32')
    IoU_2 = BatchIoU(ori_shapes.copy(), res)
    print(IoU1.mean() - IoU_2.mean())
    assert abs(IoU1.mean() - IoU_2.mean()) < 0.1, 'IoUs are not matched'
    
    # save results
    save_file = os.path.join(opt.save_path.format(tp), 'shapes.h5')
    f = h5py.File(save_file, 'w')
    f['data'] = gen_shapes
    f['pgms'] = pgms
    f['params'] = params
    f.close()
    '''
    # Interpreting programs to understandable program strings
    interpreter = Interpreter(translate, rotate, end)
    num_programs = gen_shapes.shape[0]
    for i in range(min(num_programs, opt.num_render)):
        program = interpreter.interpret(pgms[i], params[i])
        save_file = os.path.join(opt.prog_save_path.format(tp), '{}.txt'.format(i))
        print(save_file)
        with open(save_file, 'w') as out:
            out.write(program)
    
    return IoU1.mean(),IoU2.mean()

========= arguments =========
model                model/generator of GA on shapenet {}
data                 ./data/{}_testing.h5
save_path            ./output/{}/
batch_size           32
num_workers          4
info_interval        10
save_prog            False
save_img             False
num_render           10
prog_save_path       ./output/{}/programs
imgs_save_path       ./output/{}/images
========= arguments =========


In [ ]:
tp_ls = ["chair","table","bed","sofa","cabinet","bench"]
results = pd.DataFrame(columns=["chair","table","bed","sofa","cabinet","bench"],
                       index=["shape2prog w/o GA","shape2progs"])
for tp in tp_ls:
    if not os.path.isdir(opt.prog_save_path.format(tp)):
        os.makedirs(opt.prog_save_path.format(tp))
    if not os.path.isdir(opt.imgs_save_path.format(tp)):
        os.makedirs(opt.imgs_save_path.format(tp))
    model_path = opt.model.format(tp);
    data_path = opt.data.format(tp)
    opt.save_path = opt.save_path.format(tp) 
    save_path = opt.imgs_save_path.format(tp)
    IoU1 = test_on_shape_net(tp,model_path,data_path,save_path,12)
    results[tp][0] = IoU1[1]
    results[tp][1]= IoU1[0]


In [5]:
results

,chair,table,bed,sofa,cabinet,bench
shape2prog w/o GA,0.491691,0.455689,0.314078,0.371111,0.327024,0.201265
shape2progs,0.552299,0.626577,0.405465,0.628723,0.548531,0.463773
